In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import torch

from tqdm import tqdm
from annoy import AnnoyIndex

from kagglehub import KaggleDatasetAdapter
import kagglehub
import joblib

file_path = "movies_metadata.csv"

df_data = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "rounakbanik/the-movies-dataset",
  file_path,
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



/tmp/ipykernel_203704/1995972593.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_203704/1995972593.py:16: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_data = kagglehub.load_dataset(
/home/n7student/miniforge3/envs/opti_sto/lib/python3.11/site-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(
/home/n7student/miniforge3/envs/opti_sto/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initializa

In [2]:
print("Overview NaN :", (df_data.overview.isna().sum()))
df_data = df_data.dropna(subset="overview")
df_data = df_data[["overview", "title"]]

#split data
df_train, df_test = train_test_split(df_data, test_size=0.5, random_state=42)

Overview NaN : 954


## Bag of Words

In [3]:
vectorizer = CountVectorizer(stop_words='english')
train_count_matrix = vectorizer.fit_transform(df_train.overview)

len(vectorizer.vocabulary_)

53363

In [4]:
# save vectorizer
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [5]:
train_count_matrix.shape

(22256, 53363)

## Annoy

In [6]:
#annoy
annoy_index = AnnoyIndex(len(vectorizer.vocabulary_), 'angular')

for i in tqdm(range(train_count_matrix.shape[0])):
    annoy_index.add_item(i, train_count_matrix[i].toarray().flatten())

annoy_index.build(10)

100%|██████████| 22256/22256 [01:02<00:00, 357.50it/s]


True

In [7]:
def search(query_vector, k=5):
    indices = annoy_index.get_nns_by_vector(query_vector, k)
    titles = df_train['title'].iloc[indices]  
    return titles

index = 10
query_vector = train_count_matrix.toarray()[index]
result = search(query_vector)

In [8]:
df_train.iloc[index]["title"]

'The Aristocats'

In [9]:
result

608      The Aristocats
27572     The Black Cat
26174      Swedenhielms
19651           Transit
40065     You Lucky Dog
Name: title, dtype: object

In [11]:
annoy_index.save('annoy_index_bag_of_words.ann')
# save the dataframe with name of movies only
df_data[['title']].to_csv('plot_titles.csv', index=False)

In [ ]:
# load the index
annoy_index = AnnoyIndex(len(vectorizer.vocabulary_), 'angular')

## NEURAL NETWORK

## Annoy